## Problem statement
Today's Bayesian problem of the week: Suppose we visit a wild animal preserve where we know that the only animals are lions and tigers and bears, but we don't know how many of each there are.
During the tour, we see 3 lions, 2 tigers, and 1 bear. Assuming that every animal had an equal chance to appear in our sample, estimate the prevalence of each species.

What is the probability that the next animal we see is a bear?

50% lions, 33% tigers, 17% bear

In [15]:
import numpy as np
import pandas as pd

animals = ['lions', 'tigers', 'bears']
# Observations
c = np.array([3, 2, 1])
#Pseudocounts 
alphas = np.array([[1, 1, 1],
                [0.1,0.1,0.1],
                [5,5,5],
                [15,15,15],
                [20,1,20],
                  ])


In [9]:
for i in range(alphas.shape[0]):
    expected = (alphas[i] + c) / (c.sum() + alphas[i].sum())
    print(expected)

[0.44444444 0.33333333 0.22222222]
[0.49206349 0.33333333 0.17460317]
[0.38095238 0.33333333 0.28571429]
[0.35294118 0.33333333 0.31372549]
[0.4893617  0.06382979 0.44680851]


In [10]:
expected = (alphas + c) / (c.sum() + alphas.sum())
expected

array([[0.03626473, 0.02719855, 0.01813237],
       [0.02810517, 0.01903898, 0.0099728 ],
       [0.07252947, 0.06346328, 0.0543971 ],
       [0.1631913 , 0.15412511, 0.14505893],
       [0.20852221, 0.02719855, 0.19038985]])

In [11]:
import pymc3 as pm
import numpy as np

alphas = np.array([1, 1, 1])
c = np.array([3, 2, 1])

In [5]:
with pm.Model() as model:
    # Parameters of the Multinomial are from a Dirichlet
    parameters = pm.Dirichlet('parameters', a=alphas, shape=3)
    # Observed data is from a Multinomial distribution
    observed_data = pm.Multinomial(
        'observed_data', n=6, p=parameters, shape=3, observed=c)

/Users/rsilvei/Envs/bayesian/lib/python3.6/site-packages/theano/tensor/basic.py:2184: UserWarning: theano.tensor.round() changed its default from `half_away_from_zero` to `half_to_even` to have the same default as NumPy. Use the Theano flag `warn.round=False` to disable this warning.
  "theano.tensor.round() changed its default from"


In [6]:
with model:
    # Sample from the posterior
    trace = pm.sample(draws=1000, chains=2, tune=500, 
                      discard_tuned_samples=True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 4 jobs)
NUTS: [parameters_stickbreaking__]
100%|██████████| 1500/1500 [00:04<00:00, 367.06it/s]


In [12]:
summary = pm.summary(trace)
summary.index = animals
summary

,mean,sd,mc_error,hpd_2.5,hpd_97.5,n_eff,Rhat
lions,0.445770,0.151874,0.003828,0.171697,0.756508,1704.463340,0.999820
tigers,0.331939,0.143306,0.003194,0.077148,0.606064,2082.759651,0.999538
bears,0.222291,0.127752,0.002784,0.006546,0.461414,1926.774123,0.999705


In [13]:
trace['parameters']

array([[0.27036541, 0.56287248, 0.16676211],
       [0.73047916, 0.15590326, 0.11361758],
       [0.24982659, 0.19607675, 0.55409666],
       ...,
       [0.47544958, 0.25166829, 0.27288213],
       [0.53170756, 0.22830788, 0.23998456],
       [0.48684212, 0.30236509, 0.21079279]])

In [16]:
trace_df = pd.DataFrame(trace['parameters'], columns = animals)
trace_df.head()

,lions,tigers,bears
0,0.270365,0.562872,0.166762
1,0.730479,0.155903,0.113618
2,0.249827,0.196077,0.554097
3,0.511813,0.450338,0.037849
4,0.715747,0.231981,0.052272


In [18]:
# For probabilities use samples after burn in
pvals = trace_df.iloc[:, :3].mean(axis = 0)
dict(zip(animals, pvals))

{'lions': 0.4457695619169259,
 'tigers': 0.33193910991655157,
 'bears': 0.22229132816652156}

In [20]:
## Uncertainty
summary.iloc[:, 3:5]

,hpd_2.5,hpd_97.5
lions,0.171697,0.756508
tigers,0.077148,0.606064
bears,0.006546,0.461414


In [21]:
## Maximum a Posterior (MAP)
with model:
    # Find the maximum a posteriori estimate
    map_ = pm.find_MAP()

logp = -1.8042, ||grad|| = 1.118: 100%|██████████| 7/7 [00:00<00:00, 789.25it/s]


In [22]:
dict(zip(animals, map_['parameters']))

{'lions': 0.4999999599472017,
 'tigers': 0.3333330102024172,
 'bears': 0.16666702985038112}

## Sources
[Blob Post - Towards Data Science](https://towardsdatascience.com/estimating-probabilities-with-bayesian-modeling-in-python-7144be007815)  
[Notebook - Towards Data Science](https://github.com/WillKoehrsen/probabilistic-programming/blob/master/Estimating%20Probabilities%20with%20Bayesian%20Inference.ipynb)